In [1]:
"""Question 1"""

'Question 1'

In [2]:
import pandas as pd
import numpy as np
import operator
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import DictVectorizer
from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
import string
import nltk
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Chandrima\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Chandrima\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Chandrima\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
#a:  Parse the following data
amazon_data = pd.read_csv(r'F:/Annie/CornellMS/Semester 4/Machine Learning/Homework/HW3/sentiment labelled sentences/amazon_cells_labelled.txt', 
                          sep = ".	", header = None, names = ["review", "value"])
amazon_data.head()

C:\Users\Chandrima\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,review,value
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value",1
2,Great for the jawbone,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great,1


In [4]:
yelp_data = pd.read_csv(r'F:/Annie/CornellMS/Semester 4/Machine Learning/Homework/HW3/sentiment labelled sentences/yelp_labelled.txt', 
                          sep = ".	", header = None, names = ["review", "value"])
yelp_data.head()

C:\Users\Chandrima\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,review,value
0,Wow... Loved this place,1
1,Crust is not good,0
2,Not tasty and the texture was just nasty,0
3,Stopped by during the late May bank holiday of...,1
4,The selection on the menu was great and so wer...,1


In [5]:
imdb_data = pd.read_csv(r'F:/Annie/CornellMS/Semester 4/Machine Learning/Homework/HW3/sentiment labelled sentences/imdb_labelled.txt', 
                          sep = ".	", header = None, names = ["review", "value"])
imdb_data.head()

C:\Users\Chandrima\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


,review,value
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [6]:
print ("Amazon    |   Yelp    |   IMDB")
print (amazon_data.shape, "|", yelp_data.shape, "|", imdb_data.shape)

Amazon    |   Yelp    |   IMDB
(1000, 2) | (1000, 2) | (1000, 2)


In [7]:
# Label Balance
"""Given that the shape of all three text files includes 100 reviews with either a 0 or 1 value and we know that sum of the values are 500, we know that the labels must be balanced."""

'Given that the shape of all three text files includes 100 reviews with either a 0 or 1 value and we know that sum of the values are 500, we know that the labels must be balanced.'

In [8]:
#b: Preprocessing of the data

## Remove stopwords
def strip_punctuation(line):
    result = ""
    for c in line:
        if c not in string.punctuation:
            result += c
    return result

## Lemmatization of all the words
def lemmatize_line(line):
    lemma = WordNetLemmatizer()
    word_list = nltk.word_tokenize(line)
    lemmatized_output = ' '.join([lemma.lemmatize(w, pos='a') for w in word_list])
    word_list = nltk.word_tokenize(lemmatized_output)
    lemmatized_output = ' '.join([lemma.lemmatize(w, pos='v') for w in word_list])
    return lemmatized_output

## Strip stop words
def strip_stop_words(line):
    word_list = nltk.word_tokenize(line)
    filtered_words = ' '.join([word for word in word_list if word not in stopwords.words('english')])
    return filtered_words

In [9]:
def preprocess_data(raw_data):
    data = raw_data.apply(lambda x: x.astype(str).str.lower())
    pd.to_numeric(data['value'])
    data['review'].astype('str')    
    data['review'] = data['review'].apply(lambda x: lemmatize_line(x))
    data['review'] = data['review'].apply(lambda x: strip_stop_words(x))
    data['review'] = data['review'].apply(lambda x: strip_punctuation(x))   
    return data

In [10]:
amazon_data_preprocess = preprocess_data(amazon_data)
amazon_data_preprocess.head()

,review,value
0,way plug us unless go converter,0
1,good case excellent value,1
2,great jawbone,1
3,tie charger conversations last 45 minutesmajor...,0
4,mic great,1


In [11]:
yelp_data_preprocess = preprocess_data(yelp_data)
yelp_data_preprocess.head()

,review,value
0,wow love place,1
1,crust good,0
2,tasty texture nasty,0
3,stop late may bank holiday rick steve recommen...,1
4,selection menu great price,1


In [12]:
imdb_data_preprocess = preprocess_data(imdb_data)
imdb_data_preprocess.head()

,review,value
0,slowmoving aimless movie distress drift yo...,0
1,sure lose flat character audience nearly hal...,0
2,attempt artiness black white clever camera an...,0
3,little music anything speak,0
4,best scene movie gerardo try find song keep ru...,1


In [13]:
#c: Split train-test as 80:20 
def split_data(data):
    tr, te = train_test_split(data, test_size=0.2)
    return tr, te

In [14]:
amazon_train, amazon_test = split_data(amazon_data_preprocess)
yelp_train, yelp_test = split_data(yelp_data_preprocess)
imdb_train, imdb_test = split_data(imdb_data_preprocess)

In [15]:
train_data = pd.DataFrame()
train_data = pd.concat([amazon_train, imdb_train], ignore_index = True)
train_data = pd.concat([train_data, yelp_train], ignore_index = True)
train_data.shape

(2400, 2)

In [16]:
test_data = pd.DataFrame()
test_data = pd.concat([amazon_test, imdb_test], ignore_index = True)
test_data = pd.concat([test_data, yelp_test], ignore_index = True)
test_data.shape

(600, 2)

In [17]:
x_train, y_train = train_data['review'], train_data['value']
x_test, y_test = test_data['review'], test_data['value']
print (x_train.shape, y_train.shape, x_test.shape, y_test.shape)

(2400,) (2400,) (600,) (600,)


In [18]:
#d: build a dictionary of unique words for training set
train_vectorizer = CountVectorizer()
train_x_bow = train_vectorizer.fit_transform(x_train).todense()
test_vectorizer = CountVectorizer(vocabulary=train_vectorizer.get_feature_names())
test_x_bow = test_vectorizer.fit_transform(x_test).todense()

In [19]:
"""Above we have vectorized training set first, followed by limiting the vocubulary for test set as otherwise the test-set might contains words which will be absent in training set, and we will get incorrect feature vectors"""

'Above we have vectorized training set first, followed by limiting the vocubulary for test set as otherwise the test-set might contains words which will be absent in training set, and we will get incorrect feature vectors'

In [20]:
# Report feature vectors of any two reviews in the training set
print(x_train[4])
print(train_x_bow[4])
print(x_train[21])
print(train_x_bow[21])

echo problem  very unsatisfactor
[[0 0 0 ... 0 0 0]]
 warranty  problems reoccurebottom line  put money somewhere else  cingular support
[[0 0 0 ... 0 0 0]]


In [21]:
#e: Post-processing
train_x_bow_normal = normalize(train_x_bow, norm='l1')
test_x_bow_normal = normalize(test_x_bow, norm='l1')

In [22]:
"""Here, I have used L1 normalization because the reviews are qualitative so the rendundancy is unavoidable and there's no absolute measurement."""

"Here, I have used L1 normalization because the reviews are qualitative so the rendundancy is unavoidable and there's no absolute measurement."

In [23]:
#f: Sentiment Analysis using logistic regression and Naive-based (Gaussian, Bernoulli and Multinomial)
def sentiment_prediction(x_tr, y_tr, x_te, y_te):

    lr_clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(x_tr, y_tr)
    lr_score = lr_clf.score(x_te, y_te)
    print("Logistic regression accuracy: {}".format(lr_score))
    print("Confussion Matrix:", "\n", confusion_matrix(y_te, lr_clf.predict(x_te)))
    
    b_nb = BernoulliNB()
    b_fit = b_nb.fit(x_tr, y_tr)
    b_nb_score = b_fit.score(x_te, y_te)
    print("Accuracy of Naive Bayes Classifier with Bernoulli prior: {}".format(b_nb_score))
    print("Confussion Matrix:", "\n", confusion_matrix(y_te, b_fit.predict(x_te)))
    
    gaussian_nb = GaussianNB()
    gaussian_nb.fit(x_tr, y_tr)
    gaussian_nb_score = gaussian_nb.score(x_te, y_te)
    print("Accuracy of Naive Bayes Classifier with Gaussian prior: {}".format(gaussian_nb_score))
    print("Confussion Matrix:", "\n", confusion_matrix(y_te, gaussian_nb.predict(x_te)))
    
    return lr_score, gaussian_nb_score, b_nb_score, lr_clf.coef_

In [24]:
lr_score, gaussian_nb_score, b_nb_score, lr_coeff = sentiment_prediction(train_x_bow_normal, y_train, 
                                                                                     test_x_bow_normal, y_test)

Logistic regression accuracy: 0.7883333333333333
Confussion Matrix: 
 [[255  66]
 [ 61 218]]
Accuracy of Naive Bayes Classifier with Bernoulli prior: 0.8133333333333334
Confussion Matrix: 
 [[262  59]
 [ 53 226]]
Accuracy of Naive Bayes Classifier with Gaussian prior: 0.6883333333333334
Confussion Matrix: 
 [[260  61]
 [126 153]]


In [25]:
vocabulary_bow = train_vectorizer.get_feature_names()
lr_coef_val = lr_coeff.tolist()[0]
weight_vector = dict(zip(vocabulary_bow, lr_coef_val))
weight_vector = sorted(weight_vector.items(), key = lambda x: x[1], reverse = True)
weight_vector

[('great', 3.342498970911084),
 ('love', 2.297285972463214),
 ('nice', 1.902211561012528),
 ('excellent', 1.7382161891503407),
 ('amaze', 1.5418266737078257),
 ('good', 1.5301828050738429),
 ('best', 1.484047481851821),
 ('delicious', 1.3159122566579229),
 ('fantastic', 1.230057723356934),
 ('well', 1.219354117628204),
 ('awesome', 1.1642286385405223),
 ('wonderful', 1.087780198567724),
 ('perfect', 1.0828387595913571),
 ('beautiful', 1.04841777935078),
 ('friendly', 1.0207149155839712),
 ('price', 0.9733547293305825),
 ('enjoy', 0.9668645452318055),
 ('comfortable', 0.9379343418781981),
 ('cool', 0.9220651320799121),
 ('film', 0.8580676348537819),
 ('recommend', 0.854493019227082),
 ('fine', 0.7862523598486569),
 ('easy', 0.7432239649822832),
 ('wo', 0.7407247807024527),
 ('always', 0.7235658578018604),
 ('please', 0.7232622469092715),
 ('family', 0.7229009882821585),
 ('every', 0.7054253166454398),
 ('also', 0.6858382085810121),
 ('game', 0.6747120824991831),
 ('sound', 0.66014616031

In [26]:
"""I see Burnoulli for NBC gives me the best result, followed by Logistic Regression and the worst is Gaussian NBC"""

'I see Burnoulli for NBC gives me the best result, followed by Logistic Regression and the worst is Gaussian NBC'

In [27]:
#g: N-gram model
train_vectorizer_2gram = CountVectorizer(ngram_range=(2, 2))
train_x_2gram = train_vectorizer_2gram.fit_transform(x_train).todense()
test_vectorizer_2gram = CountVectorizer(ngram_range=(2, 2), vocabulary=train_vectorizer_2gram.get_feature_names())
test_x_2gram = test_vectorizer_2gram.fit_transform(x_test).todense()
print(x_train[4])
print(train_x_2gram[4])
print(x_train[21])
print(train_x_2gram[21])


echo problem  very unsatisfactor
[[0 0 0 ... 0 0 0]]
 warranty  problems reoccurebottom line  put money somewhere else  cingular support
[[0 0 0 ... 0 0 0]]


In [28]:
train_x_2gram_normal = normalize(train_x_2gram, norm='l1')
test_x_2gram_normal = normalize(test_x_2gram, norm='l1')
lr_score, gaussian_nb_score, b_nb_score, lr_coeff = sentiment_prediction(train_x_2gram_normal, 
                                                                                     y_train, test_x_2gram_normal, y_test)

Logistic regression accuracy: 0.5966666666666667
Confussion Matrix: 
 [[114 207]
 [ 35 244]]
Accuracy of Naive Bayes Classifier with Bernoulli prior: 0.5966666666666667
Confussion Matrix: 
 [[109 212]
 [ 30 249]]
Accuracy of Naive Bayes Classifier with Gaussian prior: 0.6683333333333333
Confussion Matrix: 
 [[282  39]
 [160 119]]


In [29]:
vocabulary_ng = train_vectorizer_2gram.get_feature_names()
lr_coef_val = lr_coeff.tolist()[0]
weight_vector = dict(zip(vocabulary_ng, lr_coef_val))
weight_vector = sorted(weight_vector.items(), key = lambda x: x[1], reverse = True)
weight_vector

[('work great', 1.2007221477664503),
 ('highly recommend', 1.061456145145739),
 ('great phone', 0.836510713908666),
 ('food good', 0.7456916059201351),
 ('one best', 0.7403189151851477),
 ('love place', 0.7399525083544034),
 ('good price', 0.6681850208031861),
 ('great product', 0.6499576092806894),
 ('excellent product', 0.6330740446141688),
 ('easy use', 0.6326868650730371),
 ('really good', 0.5986319213213821),
 ('good product', 0.5864949812650904),
 ('love phone', 0.5723222224750062),
 ('great price', 0.5681441125462072),
 ('pretty good', 0.5567358109681317),
 ('food delicious', 0.540999000359394),
 ('great deal', 0.533278800774241),
 ('nt disappoint', 0.5203255508206126),
 ('good expect', 0.5174475081860953),
 ('happy product', 0.5174475081860953),
 ('give 10', 0.5118110186909676),
 ('food great', 0.5069500232811133),
 ('great film', 0.5003004279411716),
 ('good time', 0.49270811515309826),
 ('product price', 0.48359346468115677),
 ('waitress friendly', 0.4812538124338857),
 ('goo

In [30]:
"""For this case, I see the output of all the process is similar, but wierdly Gaussian performs slightly better"""

'For this case, I see the output of all the process is similar, but wierdly Gaussian performs slightly better'

In [31]:
#h : PCA Analysis using SVD for Bag of Word Model
p,n = np.shape(train_x_bow_normal)
cov_matrix = np.dot(train_x_bow_normal.T, train_x_bow_normal)/(p-1)
u, s, vh = np.linalg.svd(cov_matrix, full_matrices=True)

In [32]:
# PCA Analysis using SVD for N-gram model
p2,n2 = np.shape(train_x_2gram_normal)
cov_matrix2 = np.dot(train_x_2gram_normal.T, train_x_2gram_normal)/(p-1)
u2, s2, vh2 = np.linalg.svd(cov_matrix2, full_matrices=True)

In [33]:
# Dimension=10
train_x_10 = np.dot(train_x_bow_normal, u[:,:10])
test_x_10 = np.dot(test_x_bow_normal, u[:,:10])
print ("PCA dim 10 for Bag of Word Model")
lr_score, gaussian_nb_score, b_nb_score, lr_coeff = sentiment_prediction(train_x_10, y_train, test_x_10, y_test)

train_x_10_ng = np.dot(train_x_2gram_normal, u2[:,:10])
test_x_10_ng = np.dot(test_x_2gram_normal, u2[:,:10])
print ("PCA dim 10 for Ngram Model")
lr_score_ng, gaussian_nb_score_ng, b_nb_score_ng, lr_coeff_ng = sentiment_prediction(train_x_10_ng, y_train, test_x_10_ng, y_test)

PCA dim 10 for Bag of Word Model
Logistic regression accuracy: 0.635
Confussion Matrix: 
 [[234  87]
 [132 147]]
Accuracy of Naive Bayes Classifier with Bernoulli prior: 0.6066666666666667
Confussion Matrix: 
 [[219 102]
 [134 145]]
Accuracy of Naive Bayes Classifier with Gaussian prior: 0.6233333333333333
Confussion Matrix: 
 [[268  53]
 [173 106]]
PCA dim 10 for Ngram Model
Logistic regression accuracy: 0.48333333333333334
Confussion Matrix: 
 [[ 12 309]
 [  1 278]]
Accuracy of Naive Bayes Classifier with Bernoulli prior: 0.5216666666666666
Confussion Matrix: 
 [[198 123]
 [164 115]]
Accuracy of Naive Bayes Classifier with Gaussian prior: 0.55
Confussion Matrix: 
 [[315   6]
 [264  15]]


In [34]:
print ("Bag of Words")
lr_coef_val = lr_coeff.tolist()[0]
weight_vector = dict(zip(vocabulary_bow, lr_coef_val))
weight_vector = sorted(weight_vector.items(), key = lambda x: x[1], reverse = True)
print (weight_vector)

print ("N-gram model")
lr_coef_val = lr_coeff_ng.tolist()[0]
weight_vector = dict(zip(vocabulary_ng, lr_coef_val))
weight_vector = sorted(weight_vector.items(), key = lambda x: x[1], reverse = True)
print (weight_vector)

Bag of Words
[('010', 2.8577505755728434), ('1010', 1.7759771532555801), ('110', 1.42313502364502), ('15', -0.6693773261571053), ('13', -0.9502073859538739), ('12', -1.2898382254225256), ('11', -1.6560349037860573), ('10', -1.6724272216060936), ('1199', -2.353145926039954), ('100', -3.1641965961671294)]
N-gram model
[('010 grade', 1.301424693125726), ('10 movie', 1.1854864740915152), ('10 10', 0.7551715058608989), ('10 feet', 0.750863293325864), ('10 oyvey', 0.35770008633371153), ('10 minutes', 0.32908254476652155), ('10 plus', 0.26284137344038633), ('10 simply', -0.37613923757685475), ('10 save', -1.008797119197881), ('10 110', -1.6805937767361494)]


In [37]:
# Dimension=50
train_x_50 = np.dot(train_x_bow_normal, u[:,:50])
test_x_50 = np.dot(test_x_bow_normal, u[:,:50])
print ("PCA dim 50 for Bag of Word Model")
lr_score, gaussian_nb_score, b_nb_score, lr_coeff = sentiment_prediction(train_x_50, y_train, test_x_50, y_test)

train_x_50_ng = np.dot(train_x_2gram_normal, u2[:,:50])
test_x_50_ng = np.dot(test_x_2gram_normal, u2[:,:50])
print ("PCA dim 50 for Ngram Model")
lr_score_ng, gaussian_nb_score_ng, b_nb_score_ng, lr_coeff_ng = sentiment_prediction(train_x_50_ng, y_train, test_x_50_ng, y_test)

PCA dim 50 for Bag of Word Model
Logistic regression accuracy: 0.6883333333333334
Confussion Matrix: 
 [[231  90]
 [ 97 182]]
Accuracy of Naive Bayes Classifier with Bernoulli prior: 0.6366666666666667
Confussion Matrix: 
 [[242  79]
 [139 140]]
Accuracy of Naive Bayes Classifier with Gaussian prior: 0.605
Confussion Matrix: 
 [[153 168]
 [ 69 210]]
PCA dim 50 for Ngram Model
Logistic regression accuracy: 0.5016666666666667
Confussion Matrix: 
 [[ 26 295]
 [  4 275]]
Accuracy of Naive Bayes Classifier with Bernoulli prior: 0.5466666666666666
Confussion Matrix: 
 [[212 109]
 [163 116]]
Accuracy of Naive Bayes Classifier with Gaussian prior: 0.5566666666666666
Confussion Matrix: 
 [[309  12]
 [254  25]]


In [38]:
print ("Bag of Words")
lr_coef_val = lr_coeff.tolist()[0]
weight_vector = dict(zip(vocabulary_bow, lr_coef_val))
weight_vector = sorted(weight_vector.items(), key = lambda x: x[1], reverse = True)
print (weight_vector)

print ("N-gram model")
lr_coef_val = lr_coeff_ng.tolist()[0]
weight_vector = dict(zip(vocabulary_ng, lr_coef_val))
weight_vector = sorted(weight_vector.items(), key = lambda x: x[1], reverse = True)
print (weight_vector)

Bag of Words
[('010', 3.0175075444588986), ('2007', 2.1616212233553), ('20th', 1.782399676560226), ('1010', 1.7084880952585775), ('15lb', 1.5640357645893908), ('110', 1.5339136787718948), ('40min', 1.3775368316401804), ('2000', 1.17599097234076), ('34ths', 0.9874919338561612), ('350', 0.9604229230422221), ('45', 0.9537271698196184), ('5320', 0.9509793983742273), ('42', 0.9071596072727138), ('70s', 0.8205158174925946), ('1979', 0.7618382864541814), ('5020', 0.7494372122304447), ('2mp', 0.6052035005921853), ('24', 0.5303940609376162), ('2006', 0.4958316457191242), ('3o', 0.49216090494015213), ('70000', 0.3894997871463807), ('325', 0.33827299203740013), ('1995', 0.28781694644633266), ('2005', 0.22129709031873795), ('1947', 0.18220312674656752), ('20the', 0.16901724279099792), ('70', 0.11887847749019537), ('25', 0.05392323288165849), ('1948', -0.0710876072905581), ('1998', -0.1098688603137161), ('23', -0.11052739653559647), ('30', -0.1359252357881056), ('1980', -0.37910378530586036), ('216

In [39]:
# Dimension=100
train_x_100 = np.dot(train_x_bow_normal, u[:,:100])
test_x_100 = np.dot(test_x_bow_normal, u[:,:100])
print ("PCA dim 100 for Bag of Word Model")
lr_score, gaussian_nb_score, b_nb_score, lr_coeff = sentiment_prediction(train_x_100, y_train, test_x_100, y_test)

train_x_100_ng = np.dot(train_x_2gram_normal, u2[:,:100])
test_x_100_ng = np.dot(test_x_2gram_normal, u2[:,:100])
print ("PCA dim 100 for Ngram Model")
lr_score_ng, gaussian_nb_score_ng, b_nb_score_ng, lr_coeff_ng = sentiment_prediction(train_x_100_ng, y_train, test_x_100_ng, y_test)

PCA dim 100 for Bag of Word Model
Logistic regression accuracy: 0.7283333333333334
Confussion Matrix: 
 [[243  78]
 [ 85 194]]
Accuracy of Naive Bayes Classifier with Bernoulli prior: 0.6316666666666667
Confussion Matrix: 
 [[239  82]
 [139 140]]
Accuracy of Naive Bayes Classifier with Gaussian prior: 0.6266666666666667
Confussion Matrix: 
 [[152 169]
 [ 55 224]]
PCA dim 100 for Ngram Model
Logistic regression accuracy: 0.515
Confussion Matrix: 
 [[ 35 286]
 [  5 274]]
Accuracy of Naive Bayes Classifier with Bernoulli prior: 0.55
Confussion Matrix: 
 [[215 106]
 [164 115]]
Accuracy of Naive Bayes Classifier with Gaussian prior: 0.5733333333333334
Confussion Matrix: 
 [[307  14]
 [242  37]]


In [40]:
print ("Bag of Words")
lr_coef_val = lr_coeff.tolist()[0]
weight_vector = dict(zip(vocabulary_bow, lr_coef_val))
weight_vector = sorted(weight_vector.items(), key = lambda x: x[1], reverse = True)
print (weight_vector)

print ("N-gram model")
lr_coef_val = lr_coeff_ng.tolist()[0]
weight_vector = dict(zip(vocabulary_ng, lr_coef_val))
weight_vector = sorted(weight_vector.items(), key = lambda x: x[1], reverse = True)
print (weight_vector)

Bag of Words
[('010', 2.9834418998057752), ('2007', 2.167599341128274), ('absolutely', 1.8732291568286217), ('20th', 1.7393013703611717), ('1010', 1.7170232326211983), ('15lb', 1.5479557435823645), ('110', 1.518133239512729), ('40min', 1.3958184690452062), ('accessoryone', 1.2059869310839324), ('accident', 1.154519796613082), ('2000', 1.1145959270282133), ('5320', 0.973337337236965), ('34ths', 0.9682212674411931), ('45', 0.9598660273249721), ('350', 0.9509874048514296), ('42', 0.8718156847168762), ('70s', 0.8440325971574535), ('5020', 0.7878417263875597), ('910', 0.7749260965204591), ('1979', 0.7439842558805747), ('accuse', 0.7074871823765152), ('accurate', 0.6380581179634662), ('2mp', 0.5744384637057823), ('24', 0.519366714077008), ('3o', 0.4990231724183387), ('accolades', 0.49696301364022627), ('2006', 0.4856098465279725), ('accessible', 0.426715095157759), ('absolute', 0.4187971577467303), ('70000', 0.40546563909125805), ('ache', 0.3940249310070008), ('accessable', 0.365844987151247

In [41]:
#i : Algorithm review
"""1) Bag of words using NBC with Burnoulli performs best. It might be because bag of words reserved all features of words and single word could represent features better."""


'1) Bag of words using NBC with Burnoulli performs best. It might be because bag of words reserved all features of words and single word could represent features better.'

In [42]:
"""2) The reason that PCA did not work well for this dataset might be because:
The word features are relatively evenly distributed in all features, which means the directions with highest variance cannot represent most information of the original dataset. So reducing dimensions will lose considerable part of original information.
Originally the dataset has around 2000 features. Reducing them to ~100 features reduced too much information."""


'2) The reason that PCA did not work well for this dataset might be because:\nThe word features are relatively evenly distributed in all features, which means the directions with highest variance cannot represent most information of the original dataset. So reducing dimensions will lose considerable part of original information.\nOriginally the dataset has around 2000 features. Reducing them to ~100 features reduced too much information.'

In [43]:
"""3) We see that Bag of Words always perform better than N-gram model, for all cases, both with and without PCA."""

'3) We see that Bag of Words always perform better than N-gram model, for all cases, both with and without PCA.'